In [46]:
import ast, astor
import json

In [68]:
file_content = open('design.py', 'r').read()
ast_tree = ast.parse(file_content)

In [69]:
ast_tree

In [70]:
s = ast_tree.body[0].value.s

In [72]:
if s.startswith('@nni.architecture'):
    block = ast.parse(s[len('@nni.architecture')+1:])

In [67]:
{
    block_name: block_1,
    block_inputs: [x_t, h_t-1],
    block_output_aggregate: aggregate_free_outs,
    block_outputs: [h_t],

    layer_1: {
        layer_choice: [tanh, ReLU, identity, sigmoid],
        input_candidates: [images],
        input_num: 1,
        input_aggregate: null,
        outputs: [out],
    },

    layer_2: {
        layer_choice: [tanh, ReLU, identity, sigmoid],
        input_candidates: [layer_[1:1]/out],
        input_switch: choose_one,
        outputs: [out],
    },

    layer_3: {
        layer_choice: [tanh, ReLU, identity, sigmoid],
        input_candidates: [layer_[1:2]/out],
        input_switch: choose_one,
        outputs: [out],
    }
}

NameError: name 'block_name' is not defined

In [76]:
type(.keys[0].id)

str

In [78]:
block_dict = block.body[0].value
for key, value in zip(block_dict.keys, block_dict.values):
    if isinstance(value, ast.Dict):
        print(key.id)

layer_1
layer_2
layer_3


In [73]:
def foo(ast_tree):
    print('\n'.join(re.split('\n|\\\\n', astor.dump_tree(ast_tree))))

In [74]:
foo(block)

Module(
    body=[
        Expr(
            value=Dict(
                keys=[Name(id='block_name'),
                    Name(id='block_inputs'),
                    Name(id='block_output_aggregate'),
                    Name(id='block_outputs'),
                    Name(id='layer_1'),
                    Name(id='layer_2'),
                    Name(id='layer_3')],
                values=[Name(id='block_1'),
                    List(elts=[Name(id='x_t'), BinOp(left=Name(id='h_t'), op=Sub, right=Num(n=1))]),
                    Name(id='aggregate_free_outs'),
                    List(elts=[Name(id='h_t')]),
                    Dict(
                        keys=[Name(id='layer_choice'),
                            Name(id='input_candidates'),
                            Name(id='input_num'),
                            Name(id='input_aggregate'),
                            Name(id='outputs')],
                        values=[
                            List(elts=[Name(id='tanh'), Nam

In [79]:
foo(ast_tree)

Module(
    body=[
        Expr(
            value=Str(
                s='@nni.architecture
{
    block_name: block_1,
    block_inputs: [x_t, h_t-1],
    block_output_aggregate: aggregate_free_outs,
    block_outputs: [h_t],

    layer_1: {
        layer_choice: [tanh, ReLU, identity, sigmoid],
        input_candidates: [images],
        input_num: 1,
        input_aggregate: null,
        outputs: [out],
    },

    layer_2: {
        layer_choice: [tanh, ReLU, identity, sigmoid],
        input_candidates: [layer_[1:1]/out],
        input_switch: choose_one,
        outputs: [out],
    },

    layer_3: {
        layer_choice: [tanh, ReLU, identity, sigmoid],
        input_candidates: [layer_[1:2]/out],
        input_switch: choose_one,
        outputs: [out],
    }
}
')),
        Assign(targets=[Name(id='layer_1_input_candidate')],
            value=ListComp(
                elt=Subscript(
                    value=Subscript(
                        value=Subscript(value=Name(id='gr

In [82]:
None

In [84]:
ast.dump(ast.parse('''layer_1_input_candidate = [graph['layer_1']['input_candidates'][x] for x in nni.get_candidate('layer_1')]
if graph['layer_1']['input_aggregate'] is not None:
    layer_1_aggregated_output = graph['layer_1']['input_aggregate'](*layer_1_input_candidate,)
layer_1_output = graph['layer_1']['layer_choice'][nni.get_layer_choice('layer_1')](layer_1_aggregated_output)'''))

"Module(body=[Assign(targets=[Name(id='layer_1_input_candidate', ctx=Store())], value=ListComp(elt=Subscript(value=Subscript(value=Subscript(value=Name(id='graph', ctx=Load()), slice=Index(value=Str(s='layer_1')), ctx=Load()), slice=Index(value=Str(s='input_candidates')), ctx=Load()), slice=Index(value=Name(id='x', ctx=Load())), ctx=Load()), generators=[comprehension(target=Name(id='x', ctx=Store()), iter=Call(func=Attribute(value=Name(id='nni', ctx=Load()), attr='get_candidate', ctx=Load()), args=[Str(s='layer_1')], keywords=[]), ifs=[], is_async=0)])), If(test=Compare(left=Subscript(value=Subscript(value=Name(id='graph', ctx=Load()), slice=Index(value=Str(s='layer_1')), ctx=Load()), slice=Index(value=Str(s='input_aggregate')), ctx=Load()), ops=[IsNot()], comparators=[NameConstant(value=None)]), body=[Assign(targets=[Name(id='layer_1_aggregated_output', ctx=Store())], value=Call(func=Subscript(value=Subscript(value=Name(id='graph', ctx=Load()), slice=Index(value=Str(s='layer_1')), ctx

In [85]:
astor.to_source("Module(body=[Assign(targets=[Name(id='layer_1_input_candidate', ctx=Store())], value=ListComp(elt=Subscript(value=Subscript(value=Subscript(value=Name(id='graph', ctx=Load()), slice=Index(value=Str(s='layer_1')), ctx=Load()), slice=Index(value=Str(s='input_candidates')), ctx=Load()), slice=Index(value=Name(id='x', ctx=Load())), ctx=Load()), generators=[comprehension(target=Name(id='x', ctx=Store()), iter=Call(func=Attribute(value=Name(id='nni', ctx=Load()), attr='get_candidate', ctx=Load()), args=[Str(s='layer_1')], keywords=[]), ifs=[], is_async=0)])), If(test=Compare(left=Subscript(value=Subscript(value=Name(id='graph', ctx=Load()), slice=Index(value=Str(s='layer_1')), ctx=Load()), slice=Index(value=Str(s='input_aggregate')), ctx=Load()), ops=[IsNot()], comparators=[NameConstant(value=None)]), body=[Assign(targets=[Name(id='layer_1_aggregated_output', ctx=Store())], value=Call(func=Subscript(value=Subscript(value=Name(id='graph', ctx=Load()), slice=Index(value=Str(s='layer_1')), ctx=Load()), slice=Index(value=Str(s='input_aggregate')), ctx=Load()), args=[Starred(value=Name(id='layer_1_input_candidate', ctx=Load()), ctx=Load())], keywords=[]))], orelse=[]), Assign(targets=[Name(id='layer_1_output', ctx=Store())], value=Call(func=Subscript(value=Subscript(value=Subscript(value=Name(id='graph', ctx=Load()), slice=Index(value=Str(s='layer_1')), ctx=Load()), slice=Index(value=Str(s='layer_choice')), ctx=Load()), slice=Index(value=Call(func=Attribute(value=Name(id='nni', ctx=Load()), attr='get_layer_choice', ctx=Load()), args=[Str(s='layer_1')], keywords=[])), ctx=Load()), args=[Name(id='layer_1_aggregated_output', ctx=Load())], keywords=[]))])")

AttributeError: No defined handler for node of type str